In [1]:
import requests
import pandas as pd
import datetime as dt
import subprocess
import json
import os
import re
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

In [10]:
def construct_new_ref(old_file_path, new_file_path, kind):
    old_master_df = pd.read_csv(old_file_path)
    new_df = pd.read_csv(new_file_path)
    yrs = [x for x in new_df['yr'].values]
    mos = [x for x in new_df['mo'].values]
    dys = [x for x in new_df['day'].values]
    new_df['yr_mo_day'] = [dt.datetime(yrs[i], mos[i], dys[i]) for i in range(len(yrs))]
    
    # Construction of new ref
    new_master = old_master_df.append(new_df).groupby('timestamp').max().reset_index()
    yrs = [x for x in new_master['yr'].values]
    mos = [x for x in new_master['mo'].values]
    dys = [x for x in new_master['day'].values]
    new_master['yr_mo_day'] = [dt.datetime(yrs[i], mos[i], dys[i]) for i in range(len(yrs))]
    
    today_dt = dt.datetime.today()
    today = '_'.join(str(today_dt).split(' '))
    today = '-'.join(str(today.split('.')[0]).split(':'))
    file_name = 'temp/updated_' + kind + '_' + today
    csv_string = new_master.to_csv(file_name + '.csv', index=False)

In [11]:
# Clones
old_clones = 'temp/updated_clones.csv'
new_clones = 'ncappzoo_clones_2019-10-19_18-13-53/ncappzoo_clones_2019-10-19_18-13-53.csv'
construct_new_ref(old_clones, new_clones, 'clones')

# Views
old_views = '...'
new_views = '...'
construct_new_ref(old_views, new_views, 'views')